In [6]:
!pip install ollama

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


### Data Loading

In [3]:
import wikipedia

def get_wiki_article(title):
    try:
        page = wikipedia.page(title)
        with open('jamaica.txt', 'w') as f:
            f.write(page.content)
        print("Article successfully written article to text")
    except Exception as e:
        print(f"An error occurred: {e}")

get_wiki_article('Jamaica')

with open('jamaica.txt', 'r') as file:
    text_data = file.read()

Article successfully written article to text


### Chunking

In [4]:
# Recursive Text Splitter

import nltk

nltk.download("punkt")
from nltk.tokenize import sent_tokenize
import re


def recursive_text_splitter(text, max_chunk_length=1000, overlap=100):
    """
    Helper function for chunking text recursively
    """
    # Initialize result
    result = []

    current_chunk_count = 0
    separator = ["\n", " "]
    _splits = re.split(f"({separator})", text)
    splits = [_splits[i] + _splits[i + 1] for i in range(1, len(_splits), 2)]

    for i in range(len(splits)):
        if current_chunk_count != 0:
            chunk = "".join(
                splits[
                    current_chunk_count
                    - overlap : current_chunk_count
                    + max_chunk_length
                ]
            )
        else:
            chunk = "".join(splits[0:max_chunk_length])

        if len(chunk) > 0:
            result.append("".join(chunk))
        current_chunk_count += max_chunk_length

    return result

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
#split text

chunks = recursive_text_splitter(text_data, max_chunk_length=100, overlap=10)
print("Number of Chunks: ", len(chunks))

Number of Chunks:  141


### Embedding

In [48]:
# Ollama Embedding

'''
snowflake-arctic-embed:22m 	384dim
snowflake-arctic-embed:33m 	384dim
snowflake-arctic-embed:110m 	768dim
snowflake-arctic-embed:137m 	768dim
snowflake-arctic-embed:335m 	1024dim

nomic-embed-text  (137m)

mxbai-embed-large  (335m)

all-minilm:22m
all-minilm:33m
'''

from ollama import Client


def ollama_embedder(text, model='all-minilm:22m'):
    client = Client(host='http://192.168.8.116:11434')
    response = client.embeddings(
        prompt=text,
        model=model
    )
    return response["embedding"]

In [20]:
# Ollama Embedding

embeds = []
for chunk in chunks:
    embed = ollama_embedder(chunk)
    embeds.append(embed)

In [ ]:
# GPU Accelerated Embedding

from sentence_transformers import SentenceTransformer

def embedder(chunks):    
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    return model.encode(chunks)

embeds = embedder(chunks)

### Vector Store 

In [21]:
# Insert text chunks with their embeddings

import lancedb

def prepare_data(chunks, embeddings):
    """
    Helper function to prepare data to insert in LanceDB
    """
    data = []
    for chunk, embed in zip(chunks, embeddings):
        temp = {}
        temp["text"] = chunk
        temp["vector"] = embed
        data.append(temp)
    return data


def lanceDBConnection(chunks, embeddings):
    """
    LanceDB insertion
    """
    db = lancedb.connect("lance.db")
    data = prepare_data(chunks, embeddings)
    table = db.create_table(
        "jamaica_wiki",
        data=data,
        mode="overwrite",
    )
    return table


table = lanceDBConnection(chunks, embeds)

[2024-07-05T17:38:56Z WARN  lance::dataset] No existing dataset at /home/jovyan/ai/rag/jamaica/lance.db/jamaica_wiki.lance, it will be created


### Prompt Prep

In [69]:
# Query
k = 5
question = "what salt water fish are found in jamaica"

# Embed Question
query_embedding = ollama_embedder(question)

# Semantic Search
result = table.search(query_embedding).limit(5).to_list()

# Intelligent rerank - TODO

# Create context
context = [r["text"] for r in result]
# context

In [ ]:
# View Context
for r in result:
    print(f"Text >> {r['text']}")
    print(f"Distance >> {r['_distance']}\n\n")

In [63]:
# Context Prompt

base_instruction = """Your task is to understand the user question, and provide an answer using the provided contexts. Every answer you generate should have citations in this pattern  "Answer [position].", for example: "Earth is round. [1][2]," if it's relevant.

Your answers are correct, high-quality, and written by an domain expert. If the provided context does not contain the answer, simply state, "The provided context does not have the answer."
\n\n"""
question_text = "User Question: {}\n\nContexts:\n"
context_text = "[{}]\n{}\n\n"

def create_prompt(question, context):
    prompt = ''
    for i,text in enumerate(context):
        prompt += f"{context_text.format(i+1, text)}"
    prompt = base_instruction + f"{question_text.format(question)}" + prompt
    return prompt

### LLM Query

In [79]:
# local Ollama 
from openai import OpenAI


prompt = create_prompt(question, context)

client = OpenAI(
    base_url='http://192.168.8.116:11434/v1/',
    api_key='ollama',
)

# gemma2:9b-instruct-q5_K_M, llama3:8b-instruct-fp16, granite-code:20b-instruct-q8_0, 
# mistral:7b-instruct-v0.3-fp16, mixtral:latest, gemma2:27b-instruct-q5_K_M
# codestral:latest
chat_completion = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {'role': 'user', 'content': prompt, }
    ],
    model='codestral:latest',
    temperature=0,
)
print(chat_completion.choices[0].message.content)


 Jamaican waters contain considerable resources of saltwater fish. The chief varieties of saltwater fish found in Jamaica include kingfish, jack, mackerel, whiting, bonito, and tuna. [1]


In [80]:
from openai import OpenAI

# llm
prompt = create_prompt(question, context)

client = OpenAI(api_key="sk-")
response = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {'role': 'user', 'content': prompt, }
    ],
    model='gpt-4-turbo-2024-04-09',
    temperature=0,
)

print(response.choices[0].message.content)

The chief varieties of saltwater fish found in Jamaica include kingfish, jack, mackerel, whiting, bonito, and tuna. Additionally, fish that occasionally enter freshwater and estuarine environments in Jamaica are snook, jewfish, mangrove snapper, and mullets. [1]


In [75]:
print(prompt)

Your task is to understand the user question, and provide an answer using the provided contexts. Every answer you generate should have citations in this pattern  "Answer [position].", for example: "Earth is round. [1][2]," if it's relevant.

Your answers are correct, high-quality, and written by an domain expert. If the provided context does not contain the answer, simply state, "The provided context does not have the answer."


User Question: what salt water fish are found in jamaica

Contexts:
[1]
 on Jamaica and on a few islands in the Bahamas. In addition, many types of frogs are common on the island, especially treefrogs.
Jamaican waters contain considerable resources of fresh and saltwater fish. The chief varieties of saltwater fish are kingfish, jack, mackerel, whiting, bonito, and tuna. Fish that occasionally enter freshwater and estuarine environments include snook, jewfish, mangrove snapper, and mullets. Fish that spend the majority of their lives in Jamaica's fresh waters in